In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics opencv-python matplotlib


In [ ]:
import os

dataset_path = "/content/drive/MyDrive/helmet_yolo"

print("Isi folder Helmet_detector:", os.listdir(dataset_path))
print("Isi images/train:", os.listdir(dataset_path + "/images/train")[:5])
print("Isi labels/train:", os.listdir(dataset_path + "/labels/train")[:5])


Isi folder Helmet_detector: ['images', 'labels', 'helmet_yolo_model', 'helmet_yolo_model2', 'helmet_yolo_model3', 'helmet_yolo_model4', 'helmet_yolo_model5', 'helmet_yolo_model6', 'helmet_yolo_model7', 'data.yaml', 'helmet_yolo_model8', 'helmet_yolo_model9', 'helmet_yolo_model10']
Isi images/train: ['person1 (2).jpg', 'person1 (1).jpg', 'person1.jpg', 'person2 (2).jpg', 'person2 (1).jpg']
Isi labels/train: ['helmet1.txt', 'helmet2.txt', 'person1.txt', 'person2.txt']


In [ ]:
# creating yaml path

In [ ]:
yaml_path = "/content/drive/MyDrive/helmet_yolo/data.yaml"

yaml_content = """
train: /content/drive/MyDrive/helmet_yolo/images/train
val: /content/drive/MyDrive/helmet_yolo/images/val
nc: 2
names: ["person", "helmet"]
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("data.yaml created at:", yaml_path)


data.yaml created at: /content/drive/MyDrive/helmet_yolo/data.yaml


In [ ]:
# model train

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="/content/drive/MyDrive/helmet_yolo/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="helmet_yolo_model",
    project="/content/drive/MyDrive/helmet_yolo"
)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/helmet_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=helmet_yolo_model12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

EOFError: No data left in file

In [ ]:
from google.colab.output import eval_js
from IPython.display import Javascript, display, Image
from base64 import b64decode, b64encode
import numpy as np
import cv2
import time


In [ ]:
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    return cv2.imdecode(jpg_as_np, flags=1)

def video_frame(image):
    ret, jpeg = cv2.imencode('.jpg', image)
    return b64encode(jpeg.tobytes()).decode('ascii')


In [ ]:
display(Javascript('''
var video;
var stream;

async function startWebcam(){
  video = document.createElement('video');
  video.style.display = 'block';
  document.body.appendChild(video);

  stream = await navigator.mediaDevices.getUserMedia({video:true});
  video.srcObject = stream;
  await video.play();

  google.colab.kernel.invokeFunction('run_webcam_frame', [], {});
}

startWebcam();
'''))


<IPython.core.display.Javascript object>

In [ ]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/helmet_yolo/helmet_yolo_model7/weights/best.pt"
model = YOLO(model_path)
print("Model Loaded:", model_path)
print(model.names)


Model Loaded: /content/drive/MyDrive/helmet_yolo/helmet_yolo_model7/weights/best.pt
{0: 'person', 1: 'helmet'}


In [ ]:
print("frame ok")

helmet_count = 0
no_helmet_count = 0

def process_frame():
    global helmet_count, no_helmet_count

    # Ambil frame kamera
    js_reply = eval_js("video_frame(video)")
    if not js_reply:
        return None

    frame = js_to_image(js_reply)
    results = model(frame)[0]

    persons = []
    helmets = []

    # Reset counter per frame
    helmet_count = 0
    no_helmet_count = 0

    # Pisahkan person & helmet
    for box in results.boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if cls == 0:   # person
            persons.append((x1, y1, x2, y2))
        if cls == 1:   # helmet
            helmets.append((x1, y1, x2, y2))

    # Cek setiap orang
    for (x1, y1, x2, y2) in persons:

        # Area kepala (30% bagian atas)
        head_h = int((y2 - y1) * 0.30)
        hx1, hy1, hx2, hy2 = x1, y1, x2, y1 + head_h

        wearing = False

        # Cek apakah helm berada dalam area kepala
        for (hx1_h, hy1_h, hx2_h, hy2_h) in helmets:
            cx = (hx1_h + hx2_h) // 2
            cy = (hy1_h + hy2_h) // 2

            if (x1 < cx < x2) and (y1 < cy < y1 + head_h):
                wearing = True
                break

        # Helm = hijau, Tidak helm = merah
        color = (0,255,0) if wearing else (0,0,255)
        label = "Helmet" if wearing else "No Helmet"

        # Tambah counter
        if wearing:
            helmet_count += 1
        else:
            no_helmet_count += 1

        # Gambar bounding box area kepala
        cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 2)
        cv2.putText(frame, label, (hx1, hy1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Tampilkan counter di pojok kiri atas
    cv2.putText(frame, f"Helmet: {helmet_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.putText(frame, f"No Helmet: {no_helmet_count}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    return frame


frame ok


In [ ]:
from IPython.display import clear_output

def run_webcam_frame():
    global camera_enabled
    if camera_enabled:
        frame = process_frame()
        if frame is not None:
            clear_output(wait=True)
            display(Image(data=cv2.imencode('.jpg', frame)[1].tobytes()))
    time.sleep(0.05)

from google.colab import output
output.register_callback('run_webcam_frame', run_webcam_frame)
